In [1]:
import pandas as pd

# Nombres de los archivos parquet
archivos_parquet = ['clean_df_ur.parquet', 'clean_df_ui.parquet', 'clean_df_sg.parquet']

# Cargar el primer archivo parquet
df_ur = pd.read_parquet('DataSet/' + archivos_parquet[0])

# Cargar el segundo archivo parquet
df_ui = pd.read_parquet('DataSet/' + archivos_parquet[1])

# Cargar el tercer archivo parquet
df_sg = pd.read_parquet('DataSet/' + archivos_parquet[2])

In [ ]:
df_sg

In [2]:
def PlayTimeGenre(genero: str):
    # Unir df_ui con df_sg
    merged_df = pd.merge(df_ui, df_sg, left_on='item_id', right_on='id')

    # Filtrar por género y calcular las horas jugadas por año
    filtered_df = merged_df[merged_df['genres'].str.contains(genero)]
    hours_by_year = filtered_df.groupby('release_year')['playtime_forever'].sum()

    # Encontrar el año con más horas jugadas
    max_year = hours_by_year.idxmax()

    return {"Año de lanzamiento con más horas jugadas para Género {}".format(genero): max_year}


In [7]:
# Ejemplo de prueba para PlayTimeGenre
print(PlayTimeGenre("Action"))


{'Año de lanzamiento con más horas jugadas para Género Action': 2012.0}


In [15]:
def UserForGenre(genero: str):
    # Unir df_ui con df_sg
    merged_df = pd.merge(df_ui, df_sg, left_on='item_id', right_on='id')

    # Filtrar por género
    filtered_df = merged_df[merged_df['genres'].str.contains(genero)]

    # Encontrar el usuario con más horas jugadas (en minutos originalmente)
    top_user = filtered_df.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Horas jugadas por año para el top usuario
    # Convertimos los minutos a horas dividiendo por 60
    hours_by_year = filtered_df[filtered_df['user_id'] == top_user].groupby('release_year')['playtime_forever'].sum().reset_index()
    hours_by_year['playtime_forever'] = hours_by_year['playtime_forever'] / 60  # Convertir a horas

    return {"Usuario con más horas jugadas para Género {}".format(genero): top_user, "Horas jugadas": hours_by_year.to_dict('records')}


In [16]:
# Ejemplo de prueba para UserForGenre
print(UserForGenre("Action"))


{'Usuario con más horas jugadas para Género Action': 'Sp3ctre', 'Horas jugadas': [{'release_year': 1995.0, 'playtime_forever': 3.6166666666666667}, {'release_year': 1999.0, 'playtime_forever': 0.7333333333333333}, {'release_year': 2000.0, 'playtime_forever': 1177.4}, {'release_year': 2001.0, 'playtime_forever': 0.21666666666666667}, {'release_year': 2002.0, 'playtime_forever': 3.966666666666667}, {'release_year': 2003.0, 'playtime_forever': 127.88333333333334}, {'release_year': 2004.0, 'playtime_forever': 2123.516666666667}, {'release_year': 2005.0, 'playtime_forever': 355.65}, {'release_year': 2006.0, 'playtime_forever': 14.933333333333334}, {'release_year': 2007.0, 'playtime_forever': 1879.7333333333333}, {'release_year': 2008.0, 'playtime_forever': 3.7333333333333334}, {'release_year': 2009.0, 'playtime_forever': 1805.4333333333334}, {'release_year': 2010.0, 'playtime_forever': 1301.3833333333334}, {'release_year': 2011.0, 'playtime_forever': 2581.6}, {'release_year': 2012.0, 'playt

In [4]:
def UsersRecommend(año: int):
    # Filtrar por año y recomendación positiva/neutra
    filtered_df = df_ur[(df_ur['posted_year'] == año) & (df_ur['recommend'] == True) & (df_ur['sentiment_analysis'] > 0)]

    # Agrupar por juego y contar recomendaciones
    top_games = filtered_df['item_id'].value_counts().head(3).index.tolist()

    return [{"Puesto {}".format(i+1): game} for i, game in enumerate(top_games)]


In [11]:
# Ejemplo de prueba para UsersRecommend
print(UsersRecommend(2016))


[{'Puesto 1': 730}, {'Puesto 2': 440}, {'Puesto 3': 4000}]


In [5]:
def UsersWorstDeveloper(año: int):
    # Unir df_ur con df_sg
    merged_df = pd.merge(df_ur, df_sg, left_on='item_id', right_on='id')

    # Filtrar por año y recomendación negativa
    filtered_df = merged_df[(merged_df['posted_year'] == año) & (merged_df['recommend'] == False) & (merged_df['sentiment_analysis'] == 0)]

    # Agrupar por desarrollador y contar recomendaciones negativas
    worst_devs = filtered_df['developer'].value_counts().head(3).index.tolist()

    return [{"Puesto {}".format(i+1): dev} for i, dev in enumerate(worst_devs)]


In [13]:
# Ejemplo de prueba para UsersWorstDeveloper
print(UsersWorstDeveloper(2016))


[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Hello Games'}, {'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}]


In [6]:
def sentiment_analysis(empresa_desarrolladora: str):
    # Unir df_ur con df_sg
    merged_df = pd.merge(df_ur, df_sg, left_on='item_id', right_on='id')

    # Filtrar por desarrollador
    filtered_df = merged_df[merged_df['developer'] == empresa_desarrolladora]

    # Contar análisis de sentimiento
    sentiment_counts = filtered_df['sentiment_analysis'].value_counts().to_dict()

    return {empresa_desarrolladora: {"Negative": sentiment_counts.get(0, 0), "Neutral": sentiment_counts.get(1, 0), "Positive": sentiment_counts.get(2, 0)}}


In [14]:
# Ejemplo de prueba para sentiment_analysis
print(sentiment_analysis("Valve"))


{'Valve': {'Negative': 1609, 'Neutral': 2570, 'Positive': 5384}}
